<a href="https://colab.research.google.com/github/mickymult/ZeroKnowledgeProofs/blob/main/ZKP_ZeroKnowledgeProofs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports and Libraries**

**ecdsa:** This is a Python library that provides cryptographic functionalities, specifically elliptic curve digital signatures, which are crucial for implementing secure cryptographic schemes like Zero-Knowledge Proofs. It allows us to generate private/public key pairs and to sign and verify messages using these keys.

**ipywidgets:** This library is used to create interactive UI elements (like sliders and buttons) in Jupyter notebooks or Google Colab. It makes it possible to create a simple, interactive user interface for both borrowers and lenders.

**IPython.display:** This module from IPython is used to display widgets in the notebook.

In [ ]:
!pip install ecdsa ipywidgets

**SigningKey.generate(curve=NIST384p):** This line generates a new private key using the NIST P-384 elliptic curve.

**private_key.verifying_key:** This extracts the public key associated with the generated private key.
The function returns both the private and public keys. The private key is kept secret by the borrower, while the public key can be shared with the bank.

In [14]:
from ecdsa import SigningKey, NIST384p
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

def generate_keys():
    private_key = SigningKey.generate(curve=NIST384p)
    public_key = private_key.verifying_key
    return private_key, public_key

**categorize_score(score):** This helper function categorizes the credit score into predefined ranges.

**private_key.sign(...):** This line signs the score category using the borrower's private key. The signature serves as a 'proof' without revealing the actual score.

In [15]:
def create_proof(private_key, score):
    # Simplified proof: sign the score category
    score_category = categorize_score(score)
    proof = private_key.sign(score_category.encode())
    return proof, score_category

def categorize_score(score):
    if score >= 750:
        return "excellent"
    elif 600 <= score < 750:
        return "fair"
    elif 500 <= score < 600:
        return "poor"
    else:
        return "not qualified"

**public_key.verify(...):** This verifies the signature (proof) using the borrower's public key. If the verification is successful, it means the proof is valid.

In [16]:
def verify_proof(public_key, proof, score_category):
    try:
        return public_key.verify(proof, score_category.encode())
    except:
        return False

In [17]:
import ipywidgets as widgets
from ipywidgets import Output
from ecdsa import SigningKey, NIST384p
from IPython.display import display

# Function definitions (generate_keys, create_proof, categorize_score)

# Global variables to store public key and proof
borrower_public_key = None
borrower_proof = None
borrower_score_category = None

output_area = Output()

def borrower_submit_action(b):
    global borrower_public_key, borrower_proof, borrower_score_category
    private_key, public_key = generate_keys()
    borrower_public_key = public_key
    proof, score_category = create_proof(private_key, borrower_score_input.value)
    borrower_proof = proof
    borrower_score_category = score_category

    # Display the proof and public key
    with output_area:
        output_area.clear_output()
        print("Proof generated for category:", score_category)
        print("Public Key:", public_key.to_string().hex())
        print("Proof:", proof.hex())

borrower_score_input = widgets.IntSlider(
    value=700,
    min=300,
    max=850,
    step=1,
    description='Credit Score:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

borrower_submit_button = widgets.Button(description="Submit")
borrower_submit_button.on_click(borrower_submit_action)

display(borrower_score_input, borrower_submit_button, output_area)

IntSlider(value=700, continuous_update=False, description='Credit Score:', max=850, min=300)

Button(description='Submit', style=ButtonStyle())

Output()

In [21]:
from ecdsa import VerifyingKey, NIST384p

public_key_hex = "76d9309a3cabf03336d68fdf4ff8229a68c9eb4cd267911c578c81cce9c372a0b1ff3a14b0547211851db90358d2e469446590c2687d3aa105c046c5b89e16803c2323be570d4fabe86b07821cd7de1149d7b2ea4552610d0460149b86b02ce0"
public_key = VerifyingKey.from_string(bytes.fromhex(public_key_hex), curve=NIST384p)

proof_hex = "cd3e142f7b964a3354c5fcd817f1fa3e222b4709c19e7e3e0318c49fa50f2628da99efcc93dd596b4fd8e910f1fd1a5b039677d32b157b01d44ad0b70a81dc580461250a96a2dc48b371929152ec4fecfeed3a8dee2896ef079cc889d71ecfee"
proof = bytes.fromhex(proof_hex)


In [23]:
# Assume these are received from the borrower

score_category = "excellent" # Get from borrower

def lender_verify_action(b):
    result = verify_proof(public_key, proof, score_category)
    if result:
        print("Proof verified. Category:", score_category)
    else:
        print("Verification failed.")

lender_verify_button = widgets.Button(description="Verify")
lender_verify_button.on_click(lender_verify_action)

display(lender_verify_button)

Button(description='Verify', style=ButtonStyle())

Proof verified. Category: excellent
